# Generate Train and Test Data

In [149]:
import numpy as np
from sklearn.svm import LinearSVC
from sklearn.datasets import make_classification
X_train, y_train = make_classification(n_samples=1000, n_features=20, random_state=1425)
X_test, y_test = make_classification(n_samples=2000, n_features=20, random_state=1337)
y_train = 2*y_train-1
y_test = 2*y_test-1

# Set Params

In [141]:
Sa = 0.1
Sb = 10
S = 0.5

# Train Constrained SVM with Different Regularization Settings

In [150]:
clf = LinearSVC(penalty='l1',C=S,loss='squared_hinge',random_state=0, tol=1e-5, dual=False, max_iter=1e4)
clf.fit(X_train, y_train)
print(clf.coef_)
print(clf.intercept_)
print('Test Acc:',np.sum(clf.predict(X_test)==y_test)/2000)

[[ 5.58156142e-03  0.00000000e+00  1.41480392e-02  5.52439581e-02
   1.33598678e-02 -2.64579773e-02  6.14493466e-03 -2.69039837e-01
  -5.21789888e-02  5.73762262e-05  2.64978992e-02  2.04546529e-02
   2.31047493e-03  0.00000000e+00  4.41291691e-02  0.00000000e+00
  -1.94386188e-02  9.40541853e-01  0.00000000e+00 -4.33705537e-03]]
[-0.01737903]
Test Acc: 0.5115


In [151]:
clfa = LinearSVC(penalty='l1',C=Sa,loss='squared_hinge',random_state=0, tol=1e-5, dual=False, max_iter=1e4)
clfa.fit(X_train, y_train)
print(clfa.coef_)
print(clfa.intercept_)
print('Test Acc:',np.sum(clfa.predict(X_test)==y_test)/2000)

[[ 0.          0.          0.00606546  0.04241539  0.00125051 -0.02002052
   0.         -0.25462868 -0.03847091  0.          0.0150152   0.007039
   0.          0.          0.0323356   0.         -0.01059925  0.91573268
   0.          0.        ]]
[-0.00649671]
Test Acc: 0.505


In [152]:
clfb = LinearSVC(penalty='l1',C=Sb,loss='squared_hinge',random_state=0, tol=1e-5, dual=False, max_iter=1e4)
clfb.fit(X_train, y_train)
print(clfb.coef_)
print(clfb.intercept_)
print('Test Acc:',np.sum(clfb.predict(X_test)==y_test)/2000)

[[ 7.24195366e-03  0.00000000e+00  1.63785264e-02  5.81064274e-02
   1.67489641e-02 -2.81774799e-02  9.20904788e-03 -2.73648196e-01
  -5.54402589e-02  2.62569877e-03  2.92706652e-02  2.44573685e-02
   4.37787443e-03  2.75853472e-03  4.74482509e-02  0.00000000e+00
  -2.14933542e-02  9.47117380e-01  2.61623605e-04 -6.42252746e-03]]
[-0.02013351]
Test Acc: 0.514


# Apply Screening Rules

In [145]:
A = y_train*(X_train.dot(clfa.coef_.squeeze()))
gamma_a = 0.5*(np.linalg.norm(clfa.coef_,2)**2)
gamma_b = 0.5*(np.linalg.norm(clfb.coef_,2)**2)
c = (gamma_b-gamma_a)/gamma_a
B = np.sqrt(c*(2*gamma_a*(np.linalg.norm(X_train,2,axis=1)**2)-(X_train.dot(clfa.coef_.squeeze()))**2))
idx1 = np.where(A > 1)
idx2 = np.where((A - B) > 1)
I = np.union1d(idx1[0],idx2[0])
Idx = np.setdiff1d(np.arange(0,1000),I)
X_screen = X_train[Idx]
y_screen = y_train[Idx]

In [146]:
Idx.shape

(443,)

# Validate Using Screened Dataset

In [147]:
clfs = LinearSVC(penalty='l1',C=S,loss='squared_hinge',random_state=0, tol=1e-5, dual=False, max_iter=1e4)
clfs.fit(X_screen, y_screen)
print(clfs.coef_)
print(clfs.intercept_)
print('Test Acc:',np.sum(clfs.predict(X_test)==y_test)/2000)

[[ 5.54843897e-03  0.00000000e+00  1.41573279e-02  5.52708502e-02
   1.33647622e-02 -2.64553540e-02  6.15407381e-03 -2.69021576e-01
  -5.22237008e-02  6.00414675e-05  2.64789113e-02  2.04739956e-02
   2.30726062e-03  0.00000000e+00  4.41385207e-02  0.00000000e+00
  -1.94450033e-02  9.40505876e-01  0.00000000e+00 -4.37704199e-03]]
[-0.01737142]
Test Acc: 0.5115


## Compared with results above we can conclude that our screening rules work, i.e. those points are safely discarded from training set without affecting our results.

# Validate Other Training Subset Selection

In [148]:
clfk = LinearSVC(penalty='l1',C=S,loss='squared_hinge',random_state=0, tol=1e-5, dual=False, max_iter=1e4)
clfk.fit(X_train[0:500], y_train[0:500])
print(clfk.coef_)
print(clfk.intercept_)
print('Test Acc:',np.sum(clfk.predict(X_test)==y_test)/2000)

[[ 0.03996244  0.         -0.08600235  0.05724773  0.02165426  0.0026134
   0.10583124 -0.30106755 -0.01948299 -0.04128435 -0.03642277 -0.01885621
   0.02868497 -0.03499622 -0.00573849  0.         -0.15254133  1.26719667
  -0.06761293  0.02037816]]
[-0.0597745]
Test Acc: 0.527


## We can see this subset leads to a different result.